##### Building a Chatbot with Langchain

In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

# Initialize the Groq model
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found in environment variables")

# Initialize the chat model with better configuration
model = ChatOpenAI(
    model_name="gpt-4o",
    openai_api_key=openai_api_key,
    temperature=0,
    max_tokens=1024
)

# Create a prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Keep your responses concise and helpful."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}"),
])

# Create a chain
chain = prompt | model

# Session store for chat histories
store = {}

def get_session_history(session_id: str) -> ChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Create the chat chain with message history
chat_chain = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

C:\Users\ajaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain_core.messages import HumanMessage,SystemMessage
model.invoke([HumanMessage(content="Hi,My Nam is Ajay and I am senior data engineer")])


AIMessage(content="Hello Ajay! It's great to meet you. As a senior data engineer, you must have a lot of experience with data pipelines, ETL processes, and big data technologies. If there's anything specific you'd like to discuss or any questions you have, feel free to let me know!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 20, 'total_tokens': 77, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a4d13246c5', 'id': 'chatcmpl-ClTy4apn9pgG3AJ1uM8bVhew8BgTc', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--d5b63e9c-01d7-47ff-8909-075126efb580-0', usage_metadata={'input_tokens': 20, 'output_tokens': 57, 'total_tokens': 77, 'input_token_details

In [3]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Ajay and I am a Senior Data Engineer"),
        AIMessage(content="Nice to meet you, Ajay. As a senior data engineer, I'm sure you have a deep understanding of data infrastructure, architecture, and engineering practices. What brings you here today? Do you have any specific questions or topics you'd like to discuss related to data engineering? I'm all ears."),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="Your name is Ajay, and you are a Senior Data Engineer. If there's anything specific you'd like to discuss or ask about, feel free to let me know!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 97, 'total_tokens': 130, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a4d13246c5', 'id': 'chatcmpl-ClTy5D5PmHJNH4JPY5HpB0uwHckQK', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--64624d05-ca99-401e-93a3-49661999a84c-0', usage_metadata={'input_tokens': 97, 'output_tokens': 33, 'total_tokens': 130, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [4]:
### Message History 
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model,get_session_history)



In [5]:
# Example usage
config = {"configurable": {"session_id": "chat1"}}

# Test the chat
response = chat_chain.invoke(
    {"input": "Hi, my name is Ajay and I'm a Senior Data Engineer"},
    config=config
)
print(response.content)

Hi Ajay! It's great to meet you. As a Senior Data Engineer, you must work with a lot of interesting data projects. How can I assist you today?


In [6]:
# Continue the conversation
response = chat_chain.invoke(
    {"input": "What's my name and what do I do?"},
    config=config
)
print("AI:", response.content)

# Start a new session with a different ID
new_config = {"configurable": {"session_id": "chat2"}}

# The new session won't have any context from the previous one
response = chat_chain.invoke(
    {"input": "What's my name?"},
    config=new_config
)
print("\nNew session response:", response.content)

AI: Your name is Ajay, and you are a Senior Data Engineer.

New session response: I'm sorry, I don't have access to personal data, so I don't know your name.


In [7]:
response = with_message_history.invoke([HumanMessage(content="What is my name and what do i do")],config=config)
response

AIMessage(content='Your name is Ajay, and you are a Senior Data Engineer.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 102, 'total_tokens': 116, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a4d13246c5', 'id': 'chatcmpl-ClTy9XRAJAgRi8Df4y4D9HNd7652k', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--26aeba27-8d58-445e-8f2c-0cf7638b3588-0', usage_metadata={'input_tokens': 102, 'output_tokens': 14, 'total_tokens': 116, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [8]:
### chage the session id 
from langchain_core.messages import HumanMessage


config={"configurable":{"session_id":"chat2"}}
response = with_message_history.invoke([HumanMessage(content="What is my name ")],config=config)
response



AIMessage(content="I'm sorry, I don't have access to personal data, so I don't know your name.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 42, 'total_tokens': 60, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a4d13246c5', 'id': 'chatcmpl-ClTyACQX2F5wZ6fZkTOVAnLKGZmtr', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--c667141d-4a17-4e6a-8708-a35b59472723-0', usage_metadata={'input_tokens': 42, 'output_tokens': 18, 'total_tokens': 60, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [9]:
config1={"configurable":{"session_id":"chat2"}}
response = with_message_history.invoke([HumanMessage(content="my name is Ram")],config=config1)
response

AIMessage(content='Nice to meet you, Ram! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 72, 'total_tokens': 86, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a4d13246c5', 'id': 'chatcmpl-ClTyBZdWLELFHdpVHDdxa4WPBuMPV', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--87d6f02b-fd7c-4968-bcdd-fa6000d2060a-0', usage_metadata={'input_tokens': 72, 'output_tokens': 14, 'total_tokens': 86, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [10]:
response = with_message_history.invoke([HumanMessage(content="What is my name")],config=config)
response

AIMessage(content='You mentioned that your name is Ram. How can I help you today, Ram?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 98, 'total_tokens': 115, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a4d13246c5', 'id': 'chatcmpl-ClTyC6GCp3yGnI5u3e4zlYfXt57aq', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--69abac4c-5df4-4f75-8160-a7986b2d3c21-0', usage_metadata={'input_tokens': 98, 'output_tokens': 17, 'total_tokens': 115, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

#### Prompt Template

In [11]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant.Please answer all the questions to the best of your knowledge."),
    MessagesPlaceholder(variable_name="messages")
])
chain = prompt | model
chain

ChatPromptTemplate(input_variables=['messages'], input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.mes

In [12]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Ajay")]})

AIMessage(content='Hello Ajay! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 34, 'total_tokens': 45, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a4d13246c5', 'id': 'chatcmpl-ClTyDDYQ7ooMaVDdOiCdGbSRZCEvB', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--93e663bf-72c3-45f4-92b5-249f5fd471b0-0', usage_metadata={'input_tokens': 34, 'output_tokens': 11, 'total_tokens': 45, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [13]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Ajay")],
    config=config
)

response

AIMessage(content='Hello Ajay! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 13, 'total_tokens': 24, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a4d13246c5', 'id': 'chatcmpl-ClTyDzhqMJwZ5UyWZcVFG2dXuoh92', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--b2b5cb7a-0794-4a15-a332-aa08561e0887-0', usage_metadata={'input_tokens': 13, 'output_tokens': 11, 'total_tokens': 24, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [14]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Ajay. How can I help you further?'

#### Complex Prompt 

In [15]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant.Please answer all the questions to the best of your knowledge in {language}"),
    MessagesPlaceholder(variable_name="messages")
])
chain = prompt | model
chain

ChatPromptTemplate(input_variables=['language', 'messages'], input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langch

In [16]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Ajay")],"language":"French"})
response.content

"Bonjour Ajay ! Comment puis-je vous aider aujourd'hui ?"

In [17]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [18]:
config = {"configurable": {"session_id": "chat4"}}
response = with_message_history.invoke({"messages":[HumanMessage(content="Hi , My name is Ajay and I am a Senior Data Engineer")],"language":"French"},
config=config)
response


AIMessage(content="Bonjour Ajay, enchanté de faire votre connaissance ! En tant qu'ingénieur de données senior, vous devez avoir beaucoup d'expérience dans le domaine de la gestion et de l'analyse des données. Comment puis-je vous aider aujourd'hui ?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 43, 'total_tokens': 91, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a4d13246c5', 'id': 'chatcmpl-ClTyGIfFWQQ6umpLIga0Sqsc1i3Yk', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--947d2a4f-61b5-47a4-9e56-fcc8d10236aa-0', usage_metadata={'input_tokens': 43, 'output_tokens': 48, 'total_tokens': 91, 'input_token_details': {'audio': 0, 'cache_read': 0}

In [19]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [20]:
response.content

'आपका नाम अजय है।'

In [21]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer = trim_messages(max_tokens=30,strategy="last",token_counter=model,include_system=True,allow_partial=False,start_on="human")
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [22]:
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [23]:

from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

In [24]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. Since you haven't mentioned your favorite ice cream flavor, I don't know which one you like. If you tell me your preferences or flavors you enjoy, I might be able to suggest some options you might like!"

In [25]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

"I'm sorry, but I don't have access to previous interactions or any personal data about you. If you have a specific math problem you'd like help with, please feel free to ask, and I'll do my best to assist you!"

In [26]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [27]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"I'm sorry, but I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. Therefore, I don't know your name. If you'd like, you can tell me your name!"

In [28]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"I'm sorry, but I don't have access to previous interactions or any personal data about you. If you have a specific math problem you'd like help with, please feel free to ask, and I'll do my best to assist you!"